# Guided 
[How to build a tool-using agent with LangChain](https://cookbook.openai.com/examples/how_to_build_a_tool-using_agent_with_langchain)

In [ ]:
import datetime
import json
import openai
import os
import pandas as pd
import pinecone
import re
from tqdm.autonotebook import tqdm
from typing import List, Union
import zipfile

# Langchain imports
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate, ChatPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from langchain.schema import AgentAction, AgentFinish, HumanMessage, SystemMessage
# LLM wrapper
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI
# Conversational memory
from langchain.memory import ConversationBufferWindowMemory
# Embeddings and vectorstore
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone



In [ ]:
import notebook_secrets
api_key = notebook_secrets.pinecone_token

env = notebook_secrets.pinecone_environment
index_name = 'podcasts'
pinecone.init(api_key=notebook_secrets.pinecone_token, environment=notebook_secrets.pinecone_environment)
pinecone.whoami()

In [ ]:
# Check whether the index with the same name already exists - if so, delete it
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)
    
# Creates new index
pinecone.create_index(name=index_name, dimension=1536)
index = pinecone.Index(index_name=index_name)

# Confirm our index was created
pinecone.list_indexes()